In [1]:
#https://www.digitalocean.com/community/tutorials/a-guide-to-time-series-forecasting-with-arima-in-python-3
import pandas as pd
import numpy as np
import os
from datetime import datetime

#!pip install pypyodbc
#!pip install pyodbc

import pypyodbc
import pyodbc

import statsmodels.api as sm #Modelos estadistico y metodologías econometricas
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller #Pruebas de raíz unitaria ads,kpss,ers
from statsmodels.tsa.seasonal import seasonal_decompose #Análisis de estacionalidad

import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts

import sklearn as sk
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.stats.distributions import chi2
import seaborn as sns

import warnings
import itertools
import statsmodels.api as sm
plt.style.use('fivethirtyeight')

from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
sql_queryx = pd.read_excel (r"")
pd.options.display.float_format = '{:,.0f}'.format
sql_queryx['Infra'] = sql_queryx['Infra'].cumsum().shift(-4)
sql_queryx['Maq'] = sql_queryx['Maq'].cumsum().shift(-4)
sql_queryx.dropna(subset = ['Infra'], inplace=True)
#sql_queryx

FileNotFoundError: [Errno 2] No such file or directory: ''

### Mantenimiento e Inversión en XXXX - Modelo OLS

In [ ]:
X = sql_queryx['Infra']
y = sql_queryx['Mant_inf']
# X = sm.add_constant(X)
result1 = sm.OLS(y, X.astype(float)).fit()
print(result1.summary())

In [ ]:
prstd, iv_l, iv_u = wls_prediction_std(result1)

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(X, y, 'o', label="data")
#ax.plot(X, result1.predict(), 'y-', label="Predict")
ax.plot(X, result1.fittedvalues, 'b--.', label="OLS")
ax.plot(X, iv_u, 'r--')
ax.plot(X, iv_l, 'r--')
ax.legend(loc='best');

In [ ]:
fig, ax = plt.subplots()
fig = sm.graphics.plot_fit(result1, 0, ax=ax)
ax.set_ylabel("Infraestructura")
ax.set_xlabel("Mantenimiento")
ax.set_title("Linear Regression")
plt.show()

### Mantenimiento e Inversión enXXXX - Modelo SARIMAX

##### Estimación en Niveles

In [ ]:
model = sm.tsa.statespace.SARIMAX(endog=y, exog =X, order=(0,0,4))
result2 = model.fit()
print(result2.summary());

##### Estimación en Tasas de Crecimiento

In [ ]:
X = sql_queryx['Infra']

result = sts.adfuller(X)
print('Null Hypothesis(H0): Serie has a unit root')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
print('Numbers of lag used         : %f' % result[2])
print('Numbers of observations used: %f' % result[3])
print('Criterio de Información     : %f' % result[5])

ADF=result[0]
Critical=result[4].get('10%')
if ADF<Critical:
    print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria')
elif Critical<ADF:
    print('No se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie tiene raíz unitaria, entonces es no estacionaria')
else: print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria')

In [ ]:
y = sql_queryx['Mant_inf']

result = sts.adfuller(y)
print('Null Hypothesis(H0): Serie has a unit root')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
print('Numbers of lag used         : %f' % result[2])
print('Numbers of observations used: %f' % result[3])
print('Criterio de Información     : %f' % result[5])

ADF=result[0]
Critical=result[4].get('10%')
if ADF<Critical:
    print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria')
elif Critical<ADF:
    print('No se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie tiene raíz unitaria, entonces es no estacionaria')
else: print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria')

Ninguna de las series son estacionarias

In [ ]:

#df_logged_diff_x = sql_queryx['Infra'].pct_change()

df_logged_x = sql_queryx['Infra'].apply(lambda x : np.log(x))

df_logged_diff_x = df_logged_x - df_logged_x.shift()

ax1 = plt.subplot(121)
df_logged_diff_x.plot(figsize=(12,4), color="tab:red", title="Differenced Time-Series", ax=ax1)
ax2 = plt.subplot(122)
sql_queryx['Infra'].plot(figsize=(12,4), color="tab:red", title="Original Values", ax=ax2);

In [ ]:
#df_logged_diff = sql_queryx['Mant_inf'].pct_change()

df_logged = sql_queryx['Mant_inf'].apply(lambda x : np.log(x))

df_logged_diff = df_logged - df_logged.shift()

ax1 = plt.subplot(121)
df_logged_diff.plot(figsize=(12,4), color="tab:red", title="Differenced Time-Series", ax=ax1)
ax2 = plt.subplot(122)
sql_queryx['Mant_inf'].plot(figsize=(12,4), color="tab:red", title="Original Values", ax=ax2);

In [ ]:
db1=pd.DataFrame(df_logged_diff)
db2 = pd.DataFrame(df_logged_diff_x)
db1 = db1.iloc[1:,]
db2 = db2.iloc[1:,]
db_new=pd.concat([db1,db2], axis=1)
#db_new

In [ ]:
X = db_new['Infra']

result = sts.adfuller(X)
print('Null Hypothesis(H0): Serie has a unit root')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
print('Numbers of lag used         : %f' % result[2])
print('Numbers of observations used: %f' % result[3])
print('Criterio de Información     : %f' % result[5])

ADF=result[0]
Critical=result[4].get('10%')
if ADF<Critical:
    print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria')
elif Critical<ADF:
    print('No se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie tiene raíz unitaria, entonces es no estacionaria')
else: print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria') 

In [ ]:
y = db_new['Mant_inf']

result = sts.adfuller(y)
print('Null Hypothesis(H0): Serie has a unit root')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
print('Numbers of lag used         : %f' % result[2])
print('Numbers of observations used: %f' % result[3])
print('Criterio de Información     : %f' % result[5])

ADF=result[0]
Critical=result[4].get('10%')
if ADF<Critical:
    print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria')
elif Critical<ADF:
    print('No se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie tiene raíz unitaria, entonces es no estacionaria')
else: print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria') 

In [ ]:
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 4) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(db_new['Mant_inf'],
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue


In [ ]:
warnings.filterwarnings('ignore', message='foo bar')
model = sm.tsa.statespace.SARIMAX(endog=db_new.Mant_inf, exog =db_new.Infra, order=(0,0,1))
results = model.fit()
print(results.summary());

In [ ]:
results.plot_diagnostics(figsize=(10, 7))
plt.show()

In [ ]:
pred = results.get_prediction(start=0,  dynamic=False)
pred_ci = pred.conf_int()
#pred_ci

In [ ]:
ax = db_new['Mant_inf'].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('Mantenimiento')
plt.legend()

plt.show()

In [ ]:
db_new.Infra[0:8,]

In [ ]:
pred_uc = results.get_forecast(steps=8, exog=db_new.Infra[0:8,])

# Get confidence intervals of forecasts
pred_ci = pred_uc.conf_int()

In [ ]:
ax = y.plot(label='observed', figsize=(20, 15))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('CO2 Levels')

plt.legend()
plt.show()

In [ ]:
ti = pred.predicted_mean
db_n = pd.DataFrame(ti, columns = ['mant_rate_hat'])
db_n['FECHA'] = pd.date_range('2007-04-05', periods=57, freq='Q')
db_n['mant_rate'] = y
db_n['infra_rate'] = X
db_n['error'] = y - db_n['mant_rate_hat']
db_n['Mant_inf'] = sql_queryx['Mant_inf']
db_n['Infra'] = sql_queryx['Infra']
db_n

In [ ]:
forecast= pred_uc.predicted_mean
db1 = pd.DataFrame(forecast, columns = ['mant_rate_hat'])
db1['FECHA']= pd.date_range('2021-07-01', periods=8, freq='Q')
db1

In [ ]:
mantenimiento_infra = pd.concat([db_n, db1], axis=0)
mantenimiento_infra

In [ ]:
hola = mantenimiento_infra.copy()
hola['predi1'] = hola['Mant_inf'].shift()*np.exp(hola['mant_rate'])
hola['predi2'] = hola['predi1'].shift()*np.exp(hola['mant_rate_hat'])
hola['predi3']  = hola['predi2'].shift()*np.exp(hola['mant_rate_hat'])
hola['predi4']  = hola['predi3'].shift()*np.exp(hola['mant_rate_hat'])
hola['predi5']  = hola['predi4'].shift()*np.exp(hola['mant_rate_hat'])
hola['predi6']  = hola['predi5'].shift()*np.exp(hola['mant_rate_hat'])
hola['predi7']  = hola['predi6'].shift()*np.exp(hola['mant_rate_hat'])
hola['predi8']  = hola['predi7'].shift()*np.exp(hola['mant_rate_hat'])
hola['predi9']  = hola['predi8'].shift()*np.exp(hola['mant_rate_hat'])


from datetime import datetime
import datetime

hola['date2'] = datetime.datetime(2021, 9, 30)
hola['date3'] = datetime.datetime(2021, 12, 31)
hola['date4'] = datetime.datetime(2022, 3, 31)
hola['date5'] = datetime.datetime(2022, 6, 30)
hola['date6'] = datetime.datetime(2022, 9, 30)
hola['date7'] = datetime.datetime(2022, 12, 31)
hola['date8'] = datetime.datetime(2023, 3, 31)
hola['date9'] = datetime.datetime(2023, 6, 30)

def fecha(row):
    if row['FECHA'] == row['date2']:
        clasifico = row['predi2']
    elif row['FECHA'] == row['date3']:
        clasifico = row['predi3']
    elif row['FECHA'] == row['date4']:
        clasifico = row['predi4']
    elif row['FECHA'] == row['date5']:
        clasifico = row['predi5']
    elif row['FECHA'] == row['date6']:
        clasifico = row['predi6']
    elif row['FECHA'] == row['date7']:
        clasifico = row['predi7']
    elif row['FECHA'] == row['date8']:
        clasifico = row['predi8']
    elif row['FECHA'] == row['date9']:
        clasifico = row['predi9']
    elif row['Mant_inf'] != 0 and row['Mant_inf'] != 'nan':
        clasifico = row['Mant_inf']
    else:
        clasifico = 'no'
    return clasifico

hola['mant_level_hat'] = hola.apply(fecha, axis=1)

infraestructura = hola[['FECHA','Mant_inf','Infra','mant_rate','mant_rate_hat','infra_rate','error','mant_level_hat']]
infraestructura

In [ ]:
infraestructura.to_csv(r'G:\.shortcut-targets-by-id\1CvCex95agcRH6L6KPy5QTdP2_XJoAUNX\02. Investment (I)\7. Reportes\3. Consultas\Analitico\Inversiones\Gino\Pruebas\Infraestructura_resultados.csv', sep=';', encoding='iso-8859-1',index=False)

### Mantenimiento e Inversión en XXXX - Modelo OLS

In [ ]:
X = sql_queryx['Maq']
y = sql_queryx['Mant_maq']
# X = sm.add_constant(X)
res1 = sm.OLS(y, X.astype(float)).fit()
print(res1.summary())

In [ ]:
prstd, iv_l, iv_u = wls_prediction_std(res1)

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(X, y, 'o', label="data")
#ax.plot(X, result1.predict(), 'y-', label="Predict")
ax.plot(X, res1.fittedvalues, 'b--.', label="OLS")
ax.plot(X, iv_u, 'r--')
ax.plot(X, iv_l, 'r--')
ax.legend(loc='best');

In [ ]:
fig, ax = plt.subplots()
fig = sm.graphics.plot_fit(res1, 0, ax=ax)
ax.set_ylabel("Mantenimiento de Equipos")
ax.set_xlabel("Inversión en Equipos")
ax.set_title("Linear Regression")
plt.show()

### Mantenimiento e Inversión en XXXX - Modelo SARIMAX

##### Estimación en Niveles

In [ ]:
model = sm.tsa.statespace.SARIMAX(endog=y, exog =X, order=(0,0,4))
res3 = model.fit()
print(res3.summary());

##### Estimación en Tasa de Crecimiento

In [ ]:
X = sql_queryx['Maq']

result = sts.adfuller(X)
print('Null Hypothesis(H0): Serie has a unit root')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
print('Numbers of lag used         : %f' % result[2])
print('Numbers of observations used: %f' % result[3])
print('Criterio de Información     : %f' % result[5])

ADF=result[0]
Critical=result[4].get('10%')
if ADF<Critical:
    print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria')
elif Critical<ADF:
    print('No se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie tiene raíz unitaria, entonces es no estacionaria')
else: print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria') 

In [ ]:
y = sql_queryx['Mant_maq']

result = sts.adfuller(y)
print('Null Hypothesis(H0): Serie has a unit root')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
print('Numbers of lag used         : %f' % result[2])
print('Numbers of observations used: %f' % result[3])
print('Criterio de Información     : %f' % result[5])

ADF=result[0]
Critical=result[4].get('10%')
if ADF<Critical:
    print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria')
elif Critical<ADF:
    print('No se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie tiene raíz unitaria, entonces es no estacionaria')
else: print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria') 


In [ ]:
df_logged_x = sql_queryx['Maq'].apply(lambda x : np.log(x))

df_logged_diff_x = df_logged_x - df_logged_x.shift()

ax1 = plt.subplot(121)
df_logged_diff_x.plot(figsize=(12,4), color="tab:red", title="Differenced Time-Series", ax=ax1)
ax2 = plt.subplot(122)
sql_queryx['Infra'].plot(figsize=(12,4), color="tab:red", title="Original Values", ax=ax2);

In [ ]:
df_logged = sql_queryx['Mant_maq'].apply(lambda x : np.log(x))

df_logged_diff = df_logged- df_logged.shift()

ax1 = plt.subplot(121)
df_logged_diff.plot(figsize=(12,4), color="tab:red", title="Differenced Time-Series", ax=ax1)
ax2 = plt.subplot(122)
sql_queryx['Infra'].plot(figsize=(12,4), color="tab:red", title="Original Values", ax=ax2);

In [ ]:
df1=pd.DataFrame(df_logged_diff)
df2 = pd.DataFrame(df_logged_diff_x)
df1 = df1.iloc[1:,]
df2 = df2.iloc[1:,]
db_new1=pd.concat([df1,df2], axis=1)
#db_new1

In [ ]:
# ADF Test
X = db_new1['Maq']

result = sts.adfuller(X)
print('Null Hypothesis(H0): Serie has a unit root')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
print('Numbers of lag used         : %f' % result[2])
print('Numbers of observations used: %f' % result[3])
print('Criterio de Información     : %f' % result[5])

ADF=result[0]
Critical=result[4].get('10%')
if ADF<Critical:
    print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria')
elif Critical<ADF:
    print('No se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie tiene raíz unitaria, entonces es no estacionaria')
else: print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria') 

In [ ]:
# ADF Test
y = db_new1['Mant_maq']

result = sts.adfuller(y)
print('Null Hypothesis(H0): Serie has a unit root')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
print('Numbers of lag used         : %f' % result[2])
print('Numbers of observations used: %f' % result[3])
print('Criterio de Información     : %f' % result[5])

ADF=result[0]
Critical=result[4].get('10%')
if ADF<Critical:
    print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria')
elif Critical<ADF:
    print('No se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie tiene raíz unitaria, entonces es no estacionaria')
else: print('Se rechaza H0, de rechazar H0 la probabilidad de equivocarnos sería %f' % result[1]+'\nPor tanto: La serie no tiene raíz unitaria, entonces es estacionaria') 

In [ ]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(db_new1['Mant_maq'],
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
model = sm.tsa.statespace.SARIMAX(endog=db_new1.Mant_maq, exog =db_new1.Maq, order=(0,0,1))
results2 = model.fit()
print(results2.summary());

In [ ]:
results2.plot_diagnostics(figsize=(10, 7))
plt.show()

In [ ]:
pred = results2.get_prediction(start=0,  dynamic=False)
pred_ci = pred.conf_int()
#pred_ci


In [ ]:
ax = db_new1['Mant_maq'].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('Mantenimiento')
plt.legend()

plt.show()

In [ ]:
db_new1['mant_rate'] = db_new1['Mant_maq']
db_new1['mant_rate_hat']= pred.predicted_mean
db_new1['maq_rate'] = db_new1['Maq']
db_new1['error'] = db_new1['Mant_maq'] - db_new1['mant_rate_hat']
db_new1['Mant_maq'] = sql_queryx['Mant_maq']
db_new1['Maq'] = sql_queryx['Maq']
db_new1

In [ ]:
db_new1.to_csv(r'G:\.shortcut-targets-by-id\1CvCex95agcRH6L6KPy5QTdP2_XJoAUNX\02. Investment (I)\7. Reportes\3. Consultas\Analitico\Inversiones\Gino\Pruebas\Equipos_results.csv', sep=';', encoding='iso-8859-1')